# Tiền xử lí dữ liệu sơ bộ

Cấu trúc các trường dữ liệu và cách viết của mỗi bên không thống nhất nên chia thành 2 phần cho mỗi link

## Chotot
- Dữu liệu thu thập: title,price,Số Km đã đi,Còn hạn đăng kiểm,Xuất xứ,Tình trạng,Chính sách bảo hành,Hãng,Dòng xe,Năm sản xuất,Hộp số,Nhiên liệu,Trọng lượng,Trọng tải,location,seller,Kiểu dáng,Số chỗ,Số đời chủ,Có phụ kiện đi kèm
- Dữ liệu có thể sử dụng: price,Số Km đã đi,Xuất xứ,Tình trạng,Hãng,Dòng xe,Năm sản xuất,Nhiên liệu,Kiểu dáng,Số chỗ,Nhiên liệu

## Bonbanh
- Dữu liệu thu thập: title,Năm sản xuất,Tình trạng,Số Km đã đi,Xuất xứ,Kiểu dáng,Động cơ,Màu ngoại thất,Màu nội thất,Số chỗ ngồi,Số cửa,location,seller
- Dữ liệu có thể sử dụng:  title (model và price nằm bên trong title),Năm sản xuất,Tình trạng,Số Km đã đi,Xuất xứ,Kiểu dáng,Động cơ,Số chỗ ngồi

## Kết quả cuối
- Quy tắc trường dữ liệu: price, odometer, year, brand, model, status, origin, fuel_type, style, seats

# 0. Nguồn dữ liệu

In [ ]:
import pandas as pd

# Đọc dữ liệu từ 2 file CSV
chotot_df = pd.read_csv('../datasets/raw_chotot_car_features.csv')
bonbanh_df = pd.read_csv('../datasets/raw_bonbanh_car_features.csv')

print("Chotot shape:", chotot_df.shape)
print("Bonbanh shape:", bonbanh_df.shape)

# 1. Chuẩn hóa tên cột

In [ ]:
def extract_model_price_from_title(title):
    """
    Trích xuất model và price từ title Bonbanh:
    'Xe Toyota Fortuner Legender 2.4L 4x2 AT 2025 - 1 Tỷ 120 Triệu'

    Lấy model: từ thứ 3 đến trước dấu '-' cuối cùng
    Lấy price: phần sau dấu '-' cuối cùng
    """
    if pd.isna(title) or title.strip() == '':
        return None, None

    clean_title = ' '.join(title.split())  # loại bỏ khoảng trắng thừa

    # Tách theo dấu '-' và lấy dấu cuối cùng
    parts = clean_title.rsplit('-', 1)

    if len(parts) < 2:
        return None, None

    # Model: bỏ 2 từ đầu (Xe + hãng)
    model_words = parts[0].strip().split()
    model = ' '.join(model_words[2:]) if len(model_words) > 2 else parts[0].strip()

    # Price: phần sau dấu '-' cuối cùng
    price = parts[1].strip()

    return model, price


# Chuẩn hóa cột cho cả 2 dataset
normalized_columns = [
    'price', 'odometer', 'origin', 'status',
    'brand', 'model', 'year', 'fuel_type',
    'style', 'seats'
]

# ---- Chotot ----
chotot_selected = chotot_df[[
    'price', 'Số Km đã đi', 'Xuất xứ', 'Tình trạng',
    'Hãng', 'Dòng xe', 'Năm sản xuất', 'Nhiên liệu',
    'Kiểu dáng', 'Số chỗ'
]].copy()

chotot_selected.columns = normalized_columns

print("Chotot processed shape:", chotot_selected.shape)
print(chotot_selected.head())

# ---- Bonbanh ----
bonbanh_selected = bonbanh_df[[
    'title', 'Năm sản xuất', 'Tình trạng',
    'Số Km đã đi', 'Xuất xứ', 'Kiểu dáng',
    'Động cơ', 'Số chỗ ngồi'
]].copy()

# Trích xuất model, price, fuel_type, brand
bonbanh_selected[['model', 'price']] = bonbanh_selected['title'].apply(
    lambda x: pd.Series(extract_model_price_from_title(x))
)
bonbanh_selected['brand'] = bonbanh_selected['title'].apply(
    lambda x: ' '.join(x.split()).split()[1] if pd.notna(x) and len(' '.join(x.split()).split()) > 1 else None
)

# Đổi tên các cột còn lại và chọn theo chuẩn
bonbanh_selected = bonbanh_selected.rename(columns={
    'Năm sản xuất': 'year',
    'Tình trạng': 'status',
    'Số Km đã đi': 'odometer',
    'Xuất xứ': 'origin',
    'Động cơ': 'fuel_type',
    'Kiểu dáng': 'style',
    'Số chỗ ngồi': 'seats'
})[normalized_columns]

print("Bonbanh processed shape:", bonbanh_selected.shape)
print(bonbanh_selected.head())

# 2. Chuẩn hóa giá bán

In [ ]:
import re

def parse_vn_price(price_str):
    """
    Chuyển giá tiền từ định dạng tiếng Việt về số nguyên (đồng)
    Ví dụ:
        "2 Tỷ 185 Triệu" -> 2185000000
        "1 Tỷ" -> 1000000000
        "500 Triệu" -> 500000000
    """
    if pd.isna(price_str) or price_str.strip() == '':
        return None

    price_str = price_str.replace('.', '').replace(',', '').strip()

    total = 0

    # Tìm số Tỷ
    ty_match = re.search(r'(\d+)\s*Tỷ', price_str, flags=re.IGNORECASE)
    if ty_match:
        total += int(ty_match.group(1)) * 1_000_000_000  # 1 Tỷ = 1.000.000.000 đồng

    # Tìm số Triệu
    trieu_match = re.search(r'(\d+)\s*Triệu', price_str, flags=re.IGNORECASE)
    if trieu_match:
        total += int(trieu_match.group(1)) * 1_000_000  # 1 Triệu = 1.000.000 đồng

    # Nếu không tìm thấy Tỷ hay Triệu, thử lấy số thẳng
    if total == 0:
        digits = re.findall(r'\d+', price_str)
        if digits:
            total = int(''.join(digits))
        else:
            total = None

    return total

chotot_selected['price'] = (
    chotot_selected['price']
    .astype(str)
    .str.replace(r'\D', '', regex=True)
    .replace('', pd.NA)
    .astype('Int64')
)

bonbanh_selected['price'] = bonbanh_selected["price"].apply(parse_vn_price)


# 3. Chuẩn hóa số Kilometer đã đi

In [ ]:
# Chotot
chotot_selected['odometer'] = (
    pd.to_numeric(chotot_selected['odometer'], errors='coerce')
    .astype('Int64')
)

# Bobanh
bonbanh_selected['odometer'] = (
    bonbanh_selected['odometer']
    .astype(str)
    .str.replace(r'[^0-9]', '', regex=True)
    .pipe(pd.to_numeric, errors='coerce')
    .astype('Int64')
)

# 4. Chuẩn hóa loại nhiên liệu

In [ ]:
# Chuẩn hóa fuel_type: chỉ giữ loại nhiên liệu (Xăng, Dầu, Điện, ...), bỏ dung tích
bonbanh_selected['fuel_type'] = bonbanh_selected['fuel_type'].apply(
    lambda x: str(x).split()[0] if pd.notna(x) and str(x).strip() != '' else None
)

# 5. Chuẩn hóa số chỗ

In [ ]:
chotot_selected['seats'] = (
    chotot_selected['seats']
    .astype(str)                          # đảm bảo là string
    .str.extract(r'(\d+)')                # lấy số đầu tiên
    .astype('Int64')                       # chuyển thành số nguyên, null giữ nguyên
)

bonbanh_selected['seats'] = (
    bonbanh_selected['seats']
    .astype(str)                          # đảm bảo là string
    .str.extract(r'(\d+)')                # lấy số đầu tiên
    .astype('Int64')                       # chuyển thành số nguyên, null giữ nguyên
)

# 6. Chuẩn hóa số năm

In [ ]:
chotot_selected['year'] = (
    chotot_selected['year']
    .astype(str)                          # đảm bảo là string
    .str.extract(r'(\d+)')                # lấy số đầu tiên
    .astype('Int64')                       # chuyển thành số nguyên, null giữ nguyên
)

bonbanh_selected['year'] = (
    bonbanh_selected['year']
    .astype(str)                          # đảm bảo là string
    .str.extract(r'(\d+)')                # lấy số đầu tiên
    .astype('Int64')                       # chuyển thành số nguyên, null giữ nguyên
)

# 7. Lưu dữ liệu đã xử lý thành 1 file

In [ ]:
import os

# Tạo folder nếu chưa có
os.makedirs('../data/interim', exist_ok=True)

# Gộp 2 DataFrame
combined_df = pd.concat([chotot_selected, bonbanh_selected], ignore_index=True)

# Lưu CSV
combined_df.to_csv('../data/interim/combined_interim.csv', index=False, encoding='utf-8-sig')

del chotot_selected
del bonbanh_selected

print("✓ Đã lưu file combined_interim.csv")
print("\n" + "="*50)
print("TỔNG QUAN DỮ LIỆU")
print("="*50)
print(f"\nTổng số bản ghi: {combined_df.shape[0]}")
print(f"\nCác cột chuẩn hóa: {list(combined_df.columns)}")
